In [ ]:
import csv
import json
import pandas as pd
from collections import defaultdict 

def read_airlines(filename='airlines.dat'):
    df = pd.read_csv(filename,on_bad_lines='skip', header=None)
    df = df.dropna(axis=0, thresh = 3)
    df = df.fillna('-',axis=1)
    airlines = df.set_index(4)[1].to_dict()
    return airlines # {'AJO': 'Aeroejecutivo',

def read_airports(filename='airports.dat'):
    column_names = ["ID", "Airport Name", "City", "Country", "Code", "ICAO", 
                    "Latitude", "Longitude", "Altitude", "Timezone", "DST", 
                    "Region", "Source"]

    df = pd.read_csv('airports.dat', names = column_names, header=0)
    df = df.dropna(axis=1) #, thresh = 3)
    df = df.fillna('-',axis=1)
    airports = df.set_index('Country')["ID"].to_dict()

    return airports # {'BOA': 'Boma Airport',
    
def read_routes(filename='routes.dat'):
    routes = {}  # Map from source -> list of dests
    with open(filename) as f:
        reader = csv.reader(f)
        for line in reader:
            source, dest = line[2], line[4]
            if source not in routes:
                routes[source] = []
            routes[source].append(dest)
    return routes
        
def find_paths(routes, source, dest, max_segments):
    frontier = {source}
    seen = {source: {(source, )}}
    print(f"Finding paths from {source} to {dest} with max segments {max_segments}")
    for steps in range(max_segments):
        next_frontier = set() 
        for airport in frontier:
            # Get the destinations for the current airport
            targets = routes.get(airport, [])
            for target in targets:
                if target not in seen:
                    next_frontier.add(target)
                    seen[target] = set()
                for path in seen[airport]:
                    if len(path) == steps + 1:  
                        seen[target].add(path + (target, ))
        frontier = next_frontier
    return seen.get(dest, set())  # Ensure this returns a list of paths

def rename_path(path, airports):
    renamed_path = [airports.get(code, 'Unknown') for code in path]  # Use 'Unknown' for missing codes
    return tuple(renamed_path)

def main(source, dest, max_segments):
    airlines = read_airlines()
    airports = read_airports()
    routes = read_routes()
    paths = find_paths(routes, source, dest, max_segments)
    output = defaultdict(list)
    for path in paths: # path == tuple
        segments = len(path)-1
        renamed_path = rename_path(path, airports)
        output[segments].append(renamed_path)
    print (output)
    with open(f"{source}->{dest} (max {max_segments}).json", 'w') as f:
        json.dump(output, f, indent=2, sort_keys=True)
if __name__ == '__main__':
    main('SFO', 'BOS',2)


Finding paths from SFO to BOS with max segments 2
defaultdict(<class 'list'>, {2: [('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown'), ('Unknown', 'Unknown', 'Unknown')], 1: [('Unknown', 'Unknown')]})
